In [ ]:
#For Kaggle
#date=''

#For WL data
source='WL'
date='20241118'

saveImage=False

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import dataimport as di
import result as resultMd
reload(fd)
reload(sd)
reload(di)
reload(resultMd)

print('done')

In [ ]:
import pandas as pd

dfTrx = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
dfTrx.head(5)

# Test with default hyperparameters

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt
from datetime import datetime

then= datetime.now()
predictors = fd.getPredictors(dfTrx)

x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)

modelClf = GradientBoostingClassifier(random_state=42)
modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)
resultMd.update_time_response_result('07-sklearn.ensemble.b-boosting','GradientBoostingClassifier','1-Default', duration_in_s)
f1,mcc,roc=fd.print_scores(y_test, predsTest,'all')
resultMd.update_performance_test_result('07-sklearn.ensemble.b-boosting','GradientBoostingClassifier','1-Default', f1,mcc,roc)

fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)
fd.show_prediction_graph(modelClf, x_test,y_test)



# Default parameters conclusion
No overfitting but poor result

# Scaling choice
with GradientBoostingClassifier the scaling has no influence ?

In [ ]:
%%script false
# scaling choice

import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier

predictors = fd.getPredictors(dfTrx)
parameters={}
#parameters['max_iter']=1000

modelClf = GradientBoostingClassifier(random_state=42)
duration_in_s,f1Train,f1Test, scaler = fd.processModel(modelClf,dfTrx, predictors,[], parameters, scaler=None)
scalers = fd.getScalers()
for key in scalers:
    print(key)
    scaler=scalers.get(key)
    modelClf = AdaBoostClassifier(random_state=42)
    duration_in_s,f1Train,f1Test, scaler = fd.processModel(modelClf,dfTrx, predictors, [],parameters,scaler=scaler)

# scaler has no adding value for the result

# Check about overfitting
max_depth must be greater enough to have result <p>
but a too high value occurs overfitting

In [ ]:
#%%script false

from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import recall_score, precision_score, accuracy_score, balanced_accuracy_score, f1_score
import numpy as np


x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)
predictors = fd.getPredictors(dfTrx)
train_f1s=[]
test_f1s=[]
range= []
# too low max_depth -> no result
for max_depth in np.arange(4,14,1):
    print('max_depth',max_depth)
    # Instantiate model and k-fold
    modelClf = GradientBoostingClassifier(random_state=42,n_estimators=90, learning_rate=0.01, max_depth=max_depth)
    modelClf.fit(x_train, y_train)
    train_preds = modelClf.predict(x_train)
    train_f1 = f1_score(y_train,train_preds)

    test_preds = modelClf.predict(x_test)
    test_f1 = f1_score(y_test,test_preds)
    #fd.show_confusion_matrix(y_train, train_preds)

    print("train_f1", train_f1)
    print("test_f1 ", test_f1)
    print("dif     ", train_f1-test_f1)
    train_f1s.append(train_f1)
    test_f1s.append(test_f1)
    range.append(max_depth)
    print('-----------------------')

fd.plt_train_test(range, train_f1s, "train", test_f1s,"test")

In [ ]:
#%%script false

from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import recall_score, precision_score, accuracy_score, balanced_accuracy_score, f1_score
import numpy as np


x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)
predictors = fd.getPredictors(dfTrx)
train_f1s=[]
test_f1s=[]
range= []
# too low max_depth -> no result
for n_estimators in np.arange(10,151,10):
    print('n_estimators',n_estimators)
    # Instantiate model and k-fold
    modelClf = GradientBoostingClassifier(random_state=42,n_estimators=n_estimators, learning_rate=0.01, max_depth=8)
    modelClf.fit(x_train, y_train)
    train_preds = modelClf.predict(x_train)
    train_f1 = f1_score(y_train,train_preds)

    test_preds = modelClf.predict(x_test)
    test_f1 = f1_score(y_test,test_preds)
    #fd.show_confusion_matrix(y_train, train_preds)

    print("train_f1", train_f1)
    print("test_f1 ", test_f1)
    print("dif     ", train_f1-test_f1)
    train_f1s.append(train_f1)
    test_f1s.append(test_f1)
    range.append(n_estimators)
    print('-----------------------')

fd.plt_train_test(range, train_f1s, "train", test_f1s,"test")

# Hyperparameters tuning
starting point
max_depth between 3 and 8
n_estimators=50,
learning_rate=0.01 to avoid overfitting

In [ ]:
%%script false
from scipy.stats import randint

modelClf = GradientBoostingClassifier(random_state=42)
# starting point n_estimators=100,  max_depth=7 
dic_param={
    'n_estimators': randint(25,150),
    'max_depth': randint(4,9),
    'learning_rate':[0.01]
}
res=fd.hyperparameterSelectionRandomizedSearchCV(modelClf, dic_param, 'f1', dfTrx, predictors, [], None)
print(res)
#{'learning_rate': 2, 'n_estimators': 514}
#0.1096848223301722

In [ ]:
%%script false
modelClf = GradientBoostingClassifier(random_state=42)
 
dic_param={
    'n_estimators':[95,105,115],
    'max_depth':[7,8,9],
    'learning_rate':[0.01]
}
res=fd.hyperparameterSelectionGridSearchCV(modelClf, dic_param, 'f1', dfTrx, predictors, [], None)
print(res)

#{'learning_rate': 2, 'n_estimators': 450}
#0.1096848223301722
#scoref1 0.10342393222732087


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
import matplotlib.pyplot as plt
from datetime import datetime

then= datetime.now()
predictors = fd.getPredictors(dfTrx)

x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)

modelClf = GradientBoostingClassifier(random_state=42)
parameters={'learning_rate': 0.01, 'n_estimators':110, 'max_depth':8}
modelClf.set_params(**parameters)

modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)
resultMd.update_time_response_result('07-sklearn.ensemble.b-boosting','GradientBoostingClassifier','2-After tuning', duration_in_s)
f1,mcc,roc=fd.print_scores(y_test, predsTest,'all')
resultMd.update_performance_test_result('07-sklearn.ensemble.b-boosting','GradientBoostingClassifier','2-After tuning', f1,mcc,roc)

fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)
fd.show_prediction_graph(modelClf, x_test,y_test,'../imgs/FD07C-GradientBoostingClassifierProbaHistogram' if saveImage else None)



In [ ]:
y_pred_prob = modelClf.predict_proba(x_test)[:,1]

range = []
f1s = []
rocs = []
for threshold  in [0.025,0.05,0.1,0.15,0.2,0.3,0.4,0.5,0.6]:
    print("--- threshold --", threshold)
    y_pred_adjusted = (y_pred_prob>threshold).astype(int)
    f1,mcc,roc=fd.print_scores(y_test, y_pred_adjusted,'all')
    range.append(threshold)
    f1s.append(f1)
    rocs.append(roc)

fd.plt_train_test(range, f1s,"f1 score",rocs,"rocs score")

In [ ]:
files = fd.getAllFiles()

range = []
f1s = []
rocs = []
loop =0
for file in files:
    loop=loop+1
    range.append(loop)
    print(file)
  
    dfTrx = pd.read_csv('../data/cleaned/'+source+file)
    predictors = fd.getPredictors(dfTrx)

    preds = modelClf.predict(dfTrx[predictors])

    f1,mcc,roc= fd.print_scores(dfTrx['Class'], preds,'All', True)
    #fd.show_importance(modelClf,predictors)
    fd.show_confusion_matrix(dfTrx['Class'], preds)
    f1s.append(f1)
    rocs.append(roc)

fd.plt_train_test(range, f1s)
resultMd.update_performance_nextdays_result('07-sklearn.ensemble.b-boosting','GradientBoostingClassifier','2-After tuning', f1s[0],f1s[1],f1s[2],f1s[3],rocs[0],rocs[1],rocs[2],rocs[3])

